# Stock Price Predictor

The following will use the **MediaTek Breeze-7B** model as a base to compile 30 bullish market signals as training data.

The model will be trained as a stock price predictor using the **Direct Preference Optimization (DPO)** method.


## Install Packages

In [1]:
!pip install -q bitsandbytes==0.43.1 datasets==2.19.0 peft==0.10.0 trl==0.8.6 accelerate==0.29.3

In [2]:
import json
import torch
import pandas as pd
from datasets import Dataset
from peft import LoraConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig, GenerationConfig
from trl import DPOTrainer

## Load dataset

In [11]:
!wget "https://raw.githubusercontent.com/RyanCCJ/LLM-practice/refs/heads/master/Practice_II/stock_dataset/DPO.json"

training_df = pd.read_json('/content/DPO.json')
training_df.head()

--2024-10-18 23:04:44--  https://raw.githubusercontent.com/RyanCCJ/LLM-practice/refs/heads/master/Practice_II/stock_dataset/DPO.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8848 (8.6K) [text/plain]
Saving to: ‘DPO.json’

DPO.json            100%[===================>]   8.64K  --.-KB/s    in 0s      

2024-10-18 23:04:44 (104 MB/s) - ‘DPO.json’ saved [8848/8848]



,id,prompt,chosen,rejected
0,1,請問近期台灣股市的電子股表現如何？,電子股近期有穩定增長，主要受益於全球科技需求的提升。,電子股表現受全球市場需求影響，走勢波動，需持續觀察科技趨勢。
1,2,台灣股市近期受到什麼因素影響？,近期國際市場波動和美國升息政策影響了台灣股市的走勢。,台股表現受國際經濟、政策變動及市場情緒影響，未來走勢難以預測。
2,3,台灣股市的金融股這週表現如何？,金融股這週小幅上漲，反映出投資者對本地經濟穩定的信心。,金融股表現與國內經濟狀況相關，近期波動性高，需密切關注市場消息。
3,4,台灣股市目前適合進場嗎？,目前市場波動較大，投資者可考慮長期布局來降低風險。,市場進場時機受多種因素影響，投資者需根據個人風險承受度做決定。
4,5,台積電股價近期有變化嗎？,台積電股價近期穩定，市場看好其在先進製程上的領導地位。,台積電股價波動受全球科技趨勢影響，未來走勢需看市場需求變化。


## Load model

In [4]:
model_name = "MediaTek-Research/Breeze-7B-Instruct-v0_1"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    trust_remote_code=True,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4'
    )
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

## Before DPO training

In [36]:
question = "請問台積電股價會上漲還是下降？"

In [37]:
def data_formulate(data):
    messages = [
        {"role": "system", "content": '回覆請少於50字'},
        {"role": "user", "content": data},
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer(data_formulate(question), return_tensors="pt").to('cuda')
generation_config=GenerationConfig(
    do_sample=False,
    max_new_tokens = 200,
    pad_token_id = tokenizer.pad_token_id
)
output = model.generate(**inputs, generation_config=generation_config)
output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]

print('Question: ' + question)
print('Response: ' + output)

Question: 請問台積電股價會上漲還是下降？
Response: 台積電股價的漲跌幅取決於市場需求、競爭力、公司營運表現等因素。建議您關注財經新聞或諮詢專業投資人。


## Training

In [19]:
# prepare training data
training_df['prompt'] = training_df['prompt'].apply(data_formulate)
train_dataset = Dataset.from_pandas(training_df)

training_args = TrainingArguments(
    output_dir='./',
    per_device_train_batch_size=1,
    num_train_epochs=2,
    gradient_accumulation_steps=8,
    gradient_checkpointing=False,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps = 1,
    warmup_ratio = 0.1,
    report_to = 'none'
)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

dpo_trainer = DPOTrainer(
    model=model,
    args=training_args,
    beta=0.1,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
)

dpo_trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:300: UserWarning: `max_length` is not set in the DPOTrainer's init it will default to `512` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:307: UserWarning: `max_prompt_length` is not set in the DPOTrainer's init it will default to `128` by default, but you should do it yourself in the future.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:332: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Step,Training Loss
1,0.693100
2,0.693100
3,0.505300
4,0.328400
5,0.192700
6,0.114300


TrainOutput(global_step=6, training_loss=0.4211619434257348, metrics={'train_runtime': 153.3497, 'train_samples_per_second': 0.391, 'train_steps_per_second': 0.039, 'total_flos': 0.0, 'train_loss': 0.4211619434257348, 'epoch': 1.6})

## After DPO training

In [39]:
inputs = tokenizer(data_formulate(question), return_tensors="pt").to('cuda')
generation_config=GenerationConfig(
    do_sample=False,
    max_new_tokens = 200,
    pad_token_id = tokenizer.pad_token_id
)
output = model.generate(**inputs, generation_config=generation_config)
output = tokenizer.batch_decode(output, skip_special_tokens=True)[0].split('[/INST] ')[1]

print('Question: ' + question)
print('Response: ' + output)

Question: 請問台積電股價會上漲還是下降？
Response: 台積電股價在過去一段時間內呈現上漲趨勢，主要受到半導體產業的強勁需求和公司的穩健營運表現所驅動。然而，投資者應注意市場波動和經濟環境的影響。建議您諮詢專業投資顧問以獲得個人化的建議。
